In [71]:
import os
import pandas as pd
import numpy as np
import collections
from sklearn.preprocessing import StandardScaler


수비점수 = 수비율 * 수비이닝 * 포지션 가중치
March
포지션 가중치
- 포수 = 8
- 내야수
  = 1루수 2
  = 3루수 4
  = 2루수 6
  = 유격수 7
- 외야수
  = 좌익수 2
  = 우익수 2.5
  = 중견수 3.5
  
kWAR 포지션보정점수 
MinMaxScaler
포수 = 1.0     ## 1.0
1루수 = 0.0    ## -1.0
2루수 = 0.6    ## 0.2
3루수 = 0.6    ## 0.2
유격수 = 0.8   ## 0.6
좌익수 = 0.2   ## -0.6
중견수 = 0.4   ## -0.2
우익수 = 0.2   ## -0.6
  
FPCT (수비율or 수비성공률) - (자살+보살) / (자살+보살+실책) 

In [96]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율')
dfn_1 = pd.read_csv('2019_전체.csv')
dfn_1.index = dfn_1['선수명']
del dfn_1['선수명']
#dfn_1 = dfn_1.drop(columns = ['G', 'GS', 'PKO', 'PO', 'A', 'DP', 'PB', 'SB', 'CS', 'CS%'])
dfn_1 = dfn_1.drop(columns = ['CS%'])
dfn_1 = dfn_1.replace('-', '0')
dfn_1

,팀명,POS,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
선수명,,,,,,,,,,,,,,
강경학,한화,2루수,13,4,51.00,1,0,20,14,6,0.971,0,0,0
강경학,한화,1루수,18,5,69.00,0,0,70,3,7,1.000,0,0,0
강경학,한화,유격수,27,17,153.00,3,0,31,51,18,0.965,0,0,0
강로한,롯데,3루수,27,23,183.33,3,0,18,39,1,0.950,0,0,0
강로한,롯데,유격수,52,28,261.67,12,0,45,78,25,0.911,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황재균,KT,1루수,4,1,8.67,0,0,12,0,1,1.000,0,0,0
황재균,KT,유격수,9,6,54.00,1,0,12,18,4,0.968,0,0,0
황재균,KT,3루수,117,112,964.00,10,0,103,215,22,0.970,0,0,0


In [97]:
## integer로 타입변환
FPCT_list = []
for a in dfn_1['FPCT'] :
    FPCT_list.append(int(a.replace('.', '').strip()) / 1000)
dfn_1['FPCT'] = FPCT_list

# CS_list = []
# for b in dfn_1['CS%'] :
#     FPCT_list.append(int(b))
# dfn_1['CS'] = CS_list

dfn_1

,팀명,POS,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
선수명,,,,,,,,,,,,,,
강경학,한화,2루수,13,4,51.00,1,0,20,14,6,0.971,0,0,0
강경학,한화,1루수,18,5,69.00,0,0,70,3,7,1.000,0,0,0
강경학,한화,유격수,27,17,153.00,3,0,31,51,18,0.965,0,0,0
강로한,롯데,3루수,27,23,183.33,3,0,18,39,1,0.950,0,0,0
강로한,롯데,유격수,52,28,261.67,12,0,45,78,25,0.911,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황재균,KT,1루수,4,1,8.67,0,0,12,0,1,1.000,0,0,0
황재균,KT,유격수,9,6,54.00,1,0,12,18,4,0.968,0,0,0
황재균,KT,3루수,117,112,964.00,10,0,103,215,22,0.970,0,0,0


In [98]:
# ## 이닝 표준화
# #StandardScaler().fit_transform(df_fin)
# dfn_11 = pd.DataFrame(dfn_1['IP'])
# dfn_11
# dfn_12 = pd.DataFrame(StandardScaler().fit_transform(dfn_11), index = dfn_1.index, columns = ['s_IP'])
# dfn_1['IP'] = dfn_12['s_IP']
# dfn_1

In [99]:
dfn_1_uniq = dfn_1.reset_index()
dfn_1_uniq = dfn_1_uniq.append({'선수명' : 0},  ignore_index = True) ## 루프 돌릴 때 마지막 row 하나 더 필요해서 추가
dfn_1_uniq = dfn_1_uniq.drop(columns = ['팀명', 'POS'])
dfn_1_uniq

,선수명,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
0,강경학,13.0,4.0,51.00,1.0,0.0,20.0,14.0,6.0,0.971,0.0,0.0,0.0
1,강경학,18.0,5.0,69.00,0.0,0.0,70.0,3.0,7.0,1.000,0.0,0.0,0.0
2,강경학,27.0,17.0,153.00,3.0,0.0,31.0,51.0,18.0,0.965,0.0,0.0,0.0
3,강로한,27.0,23.0,183.33,3.0,0.0,18.0,39.0,1.0,0.950,0.0,0.0,0.0
4,강로한,52.0,28.0,261.67,12.0,0.0,45.0,78.0,25.0,0.911,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,황재균,9.0,6.0,54.00,1.0,0.0,12.0,18.0,4.0,0.968,0.0,0.0,0.0
604,황재균,117.0,112.0,964.00,10.0,0.0,103.0,215.0,22.0,0.970,0.0,0.0,0.0
605,황진수,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
606,황진수,5.0,0.0,9.33,0.0,0.0,0.0,1.0,1.0,1.000,0.0,0.0,0.0


In [100]:
## 선수 한 명의 기록 합친 값이 들어갈 df 생성
dfn_1_uniqs = pd.DataFrame(columns = dfn_1_uniq.columns)
dfn_1_uniqs['선수명'] = dfn_1_uniq['선수명'].unique()
dfn_1_uniqs

,선수명,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
0,강경학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강로한,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,강민국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강민호,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,강백호,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,황대인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,황윤호,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,황재균,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,황진수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
## 선수 한 명의 1년치 기록 합치기
ID_1 = '강경학'
row = 0
for d in range(1, len(dfn_1_uniqs.columns)) :
    c = 0
    row = 0
    for a in range(0, len(dfn_1_uniq)) :
        ID = dfn_1_uniq.iloc[a, 0]
        if ID == ID_1 :
            c += dfn_1_uniq.iloc[a, d]

        else :
            dfn_1_uniqs.iloc[row, d] = c
            row += 1
            c = 0
            c += dfn_1_uniq.iloc[a, d]
            ID_1 = dfn_1_uniq.iloc[a, 0]
dfn_1_uniqs

,선수명,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
0,강경학,58.0,0,0,0,0,0,0,0,0,0,0,0
1,강로한,117.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0
2,강민국,55.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0
3,강민호,109.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0
4,강백호,79.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,황대인,9.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0
278,황윤호,65.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0
279,황재균,130.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0
280,황진수,6.0,119.0,1026.67,11.0,0.0,127.0,233.0,27.0,2.938,0.0,0.0,0.0


In [102]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 첫 번째 칼럼 하나씩 밀어줌
G_list = dfn_1_uniqs['G'].to_list()
G_list.insert(0, 0)
G_list = G_list[:-1]
dfn_1_uniqs['G'] = G_list

In [103]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 두 번째 칼럼도 하나씩 밀어줌
ID_list = dfn_1_uniqs['선수명'].to_list()
ID_list.insert(0, 0)
ID_list = ID_list[:-1]
dfn_1_uniqs['선수명'] = ID_list

In [104]:
## 그리고 나서 첫 번째 쓸모없는 row 삭제
dfn_1_uniqs = dfn_1_uniqs.drop([0])
dfn_1_uniqs = dfn_1_uniqs.reset_index(drop = True)
dfn_1_uniqs

,선수명,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
0,강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0
1,강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0
2,강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0
3,강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0
4,강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,3.978,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0
277,황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0
278,황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0
279,황재균,130.0,119.0,1026.67,11.0,0.0,127.0,233.0,27.0,2.938,0.0,0.0,0.0


In [105]:
dfn_1_uniqs

,선수명,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
0,강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0
1,강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0
2,강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0
3,강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0
4,강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,3.978,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0
277,황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0
278,황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0
279,황재균,130.0,119.0,1026.67,11.0,0.0,127.0,233.0,27.0,2.938,0.0,0.0,0.0


In [106]:
dfn_1_uniqs.index = dfn_1_uniqs['선수명']
del dfn_1_uniqs['선수명']
dfn_1_uniqs

,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS
선수명,,,,,,,,,,,,
강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0
강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0
강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0
강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0
강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,3.978,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0
황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0
황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0


In [107]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\전처리')
df_2 = pd.read_csv('2019_타자_팀명.csv', encoding = 'cp949')
df_2.index = df_2['선수명']
del df_2['선수명']
df_2

,IP,E,March_score,kWAR_score,AHP_score,팀명
선수명,,,,,,
강경학,273.00,4.0,1468.641,147.829,39.993,한화
강로한,709.67,20.0,3910.467,449.717,105.735,롯데
강민국,245.67,9.0,1487.254,171.306,44.404,KT
강민호,842.00,8.0,6661.904,832.738,308.113,삼성
강백호,622.66,3.0,1549.272,127.421,23.109,KT
...,...,...,...,...,...,...
홍창기,52.00,0.0,136.500,13.400,2.337,LG
황대인,48.00,0.0,100.000,1.200,1.260,KIA
황윤호,297.67,6.0,1534.647,177.799,38.623,KIA


In [109]:
dfn_1_uniqs['팀명'] = df_2['팀명']
dfn_1_uniqs

,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS,팀명
선수명,,,,,,,,,,,,,
강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0,한화
강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0,롯데
강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0,KT
강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0,삼성
강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,3.978,0.0,0.0,0.0,KT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0,LG
황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0,KIA
황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0,KIA


In [110]:
## 수비위치 몇개까지 멀티를 했는지 count
counts = collections.Counter(dfn_1_uniq['선수명'])
df_cnt = pd.DataFrame()
df_cnt.index = counts.keys()
df_cnt['포지션 수'] = counts.values()
df_cnt = df_cnt[:-1]
df_cnt

,포지션 수
강경학,3
강로한,3
강민국,3
강민호,1
강백호,5
...,...
홍창기,3
황대인,2
황윤호,3
황재균,3


In [111]:
## df에 추가
dfn_1_uniqs['포지션 수'] = df_cnt['포지션 수']
dfn_1_uniqs

,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS,팀명,포지션 수
선수명,,,,,,,,,,,,,,
강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,2.936,0.0,0.0,0.0,한화,3
강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,2.834,0.0,0.0,0.0,롯데,3
강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,2.734,0.0,0.0,0.0,KT,3
강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.989,3.0,63.0,23.0,삼성,1
강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,3.978,0.0,0.0,0.0,KT,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,3.0,0.0,0.0,0.0,LG,3
황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,2.0,0.0,0.0,0.0,KIA,2
황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,2.9,0.0,0.0,0.0,KIA,3


In [114]:
FP_list = []
for fp in range(0, len(dfn_1_uniqs)) :
    bm = dfn_1_uniqs.iloc[fp, 5] + dfn_1_uniqs.iloc[fp, 6]
    bj = bm + dfn_1_uniqs.iloc[fp, 3]
    FP_list.append(bm/bj)
dfn_1_uniqs['FPCT'] = FP_list
dfn_1_uniqs

<ipython-input-114-c8c2f714f0f5>:5: RuntimeWarning: invalid value encountered in double_scalars
  FP_list.append(bm/bj)


,G,GS,IP,E,PKO,PO,A,DP,FPCT,PB,SB,CS,팀명,포지션 수
선수명,,,,,,,,,,,,,,
강경학,58.0,26.0,273.0,4.0,0.0,121.0,68.0,31.0,0.979275,0.0,0.0,0.0,한화,3
강로한,117.0,84.0,709.67,20.0,0.0,123.0,235.0,51.0,0.947090,0.0,0.0,0.0,롯데,3
강민국,55.0,26.0,245.67,9.0,0.0,51.0,81.0,10.0,0.936170,0.0,0.0,0.0,KT,3
강민호,109.0,104.0,842.0,8.0,1.0,660.0,48.0,7.0,0.988827,3.0,63.0,23.0,삼성,1
강백호,79.0,76.0,622.66,3.0,0.0,140.0,6.0,1.0,0.979866,0.0,0.0,0.0,KT,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
홍창기,20.0,2.0,52.0,0.0,0.0,12.0,1.0,0.0,1.000000,0.0,0.0,0.0,LG,3
황대인,9.0,5.0,48.0,0.0,0.0,40.0,4.0,6.0,1.000000,0.0,0.0,0.0,KIA,2
황윤호,65.0,30.0,297.67,6.0,0.0,49.0,81.0,14.0,0.955882,0.0,0.0,0.0,KIA,3


In [116]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율\전처리')
dfn_1_uniqs.to_csv('2019_전포지션_수비지표_합계.csv', encoding = 'cp949')